In [84]:
# Step 0 - Install Libraries

!pip install selenium pandas beautifulsoup4



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\nurul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [85]:
# Step 1 – Import libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [86]:
# Step 2 – Setup WebDriver
options = Options()
# options.add_argument('--headless')  # Aktifkan kalau mau tanpa buka browser
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(
    service=Service("C:/Users/nurul/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"),
    options=options
)


In [87]:
# Step 3 – Kategori yang diambil (tech-related)
kategori_links = {
    "Data & Product": "https://dealls.com/?loker=data-and-product",
    "Design & Creative": "https://dealls.com/?loker=design-and-creative",
    "IT & Engineering": "https://dealls.com/?loker=it-and-engineering",
    "Marketing & Social Media": "https://dealls.com/?loker=marketing-and-social-media"
}

all_data = []

In [88]:
for kategori, url in kategori_links.items():
    print(f"\n🌐 Memproses kategori: {kategori}")
    driver.get(url)
    time.sleep(2)

    # Step 4 – Klik tombol "Lebih Banyak"
    clicks = 0
    max_clicks = 25
    while clicks < max_clicks:
        try:
            load_more_btn = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//button[.//span[contains(text(), 'Lebih Banyak')]]"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_btn)
            time.sleep(1)
            load_more_btn.click()
            clicks += 1
            print(f"🔁 Klik 'Lebih Banyak' ke-{clicks}")
            time.sleep(2)
        except Exception as e:
            print(f"✅ Selesai klik atau tombol tidak ditemukan: {e}")
            break

    # Step 5 – Ambil semua card job
    job_cards = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/loker/')]")

    for card in job_cards:
        try:
            job_title_element = card.find_element(By.XPATH, ".//h2 | .//div[contains(@class,'font-bold')]")
            job_title = job_title_element.text.strip()
        except:
            continue

        detail_link = card.get_attribute("href")

        try:
            company = card.find_element(By.XPATH, ".//div[contains(@class,'gap-[4.6px]')]").text.strip().split("\n")[0]
        except:
            company = ""

        try:
            work_type = card.find_element(By.XPATH, ".//div[contains(@class,'text-tertiary-violet-50')]").text.strip()
        except:
            work_type = ""

        try:
            infos = card.find_elements(By.XPATH, ".//div[contains(@class,'text-neutral-100')]//span")
            work_mode = infos[0].text.strip() if len(infos) > 0 else ""
            experience = infos[1].text.strip() if len(infos) > 1 else ""
            salary = infos[2].text.strip() if len(infos) > 2 else ""
        except:
            work_mode = experience = salary = ""

        all_data.append({
            "Kategori": kategori,
            "Job Title": job_title,
            "Company": company,
            "Work Type": work_type,
            "Work Mode": work_mode,
            "Experience": experience,
            "Salary": salary,
            "Detail Link": detail_link
        })

print(f"\n📌 Total job dikumpulkan: {len(all_data)}")



🌐 Memproses kategori: Data & Product
🔁 Klik 'Lebih Banyak' ke-1
🔁 Klik 'Lebih Banyak' ke-2
🔁 Klik 'Lebih Banyak' ke-3
🔁 Klik 'Lebih Banyak' ke-4
✅ Selesai klik atau tombol tidak ditemukan: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7641b6f75+76917]
	GetHandleVerifier [0x0x7ff7641b6fd0+77008]
	(No symbol) [0x0x7ff763f69dea]
	(No symbol) [0x0x7ff763fc0256]
	(No symbol) [0x0x7ff763fc050c]
	(No symbol) [0x0x7ff764013887]
	(No symbol) [0x0x7ff763fe84af]
	(No symbol) [0x0x7ff76401065c]
	(No symbol) [0x0x7ff763fe8243]
	(No symbol) [0x0x7ff763fb1431]
	(No symbol) [0x0x7ff763fb21c3]
	GetHandleVerifier [0x0x7ff76448d2ad+3051437]
	GetHandleVerifier [0x0x7ff764487903+3028483]
	GetHandleVerifier [0x0x7ff7644a589d+3151261]
	GetHandleVerifier [0x0x7ff7641d183e+185662]
	GetHandleVerifier [0x0x7ff7641d96ff+218111]
	GetHandleVerifier [0x0x7ff7641bfaf4+112628]
	GetHandleVerifier [0x0x7ff7641bfca9+113065]
	GetHandleVerifier [0x0x7ff7641a6c78+10616]
	BaseThreadInitThunk [0x0x7ffb9dd2259d+29]
	RtlUse

In [ ]:
# Step 6 – Ambil detail deskripsi & kualifikasi
job_detail_data = []

for idx, job in enumerate(all_data):
    print(f"🔍 Scraping detail {idx+1}/{len(all_data)}: {job['Job Title']}")
    try:
        driver.get(job["Detail Link"])
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//h3[contains(text(), 'Deskripsi Pekerjaan') or contains(text(),'Kualifikasi') or contains(text(),'Job Requirements')]")
            )
        )

        # Deskripsi
        desc_parts = []
        try:
            desc_blocks = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Deskripsi Pekerjaan')]/following-sibling::*")
            for elem in desc_blocks:
                if elem.tag_name == "h3":
                    break
                ps = elem.find_elements(By.TAG_NAME, "p")
                if ps:
                    desc_parts.extend([p.text.strip() for p in ps if p.text.strip()])
                else:
                    text = elem.text.strip()
                    if text:
                        desc_parts.append(text)
        except:
            pass
        description = "\n".join(desc_parts)

        # Kualifikasi
        qualif_parts = []
        try:
            qualif_blocks = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Kualifikasi') or contains(text(), 'Job Requirements')]/following-sibling::*")
            for elem in qualif_blocks:
                if elem.tag_name == "h3":
                    break
                lis = elem.find_elements(By.TAG_NAME, "li")
                if lis:
                    qualif_parts.extend([li.text.strip() for li in lis if li.text.strip()])
                else:
                    text = elem.text.strip()
                    if text:
                        qualif_parts.append(text)
        except:
            pass
        qualification = "\n".join(qualif_parts)

        job_detail_data.append({
            **job,
            "Job Description": description,
            "Qualification": qualification
        })

    except Exception as e:
        print(f"❌ Gagal ambil detail untuk {job['Job Title']} - {e}")


🔍 Scraping detail 1/1138: Product Manager [Bayarind]
🔍 Scraping detail 2/1138: Data Scientist Executive
🔍 Scraping detail 3/1138: Billing Staff
🔍 Scraping detail 4/1138: Procurement Data Analyst Lead
🔍 Scraping detail 5/1138: Process Management System Analyst
🔍 Scraping detail 6/1138: Quality & Risk Management - Senior Associate
🔍 Scraping detail 7/1138: Product - Analyst
🔍 Scraping detail 8/1138: DMS Development - Program
🔍 Scraping detail 9/1138: Data Center Operation - Specialist
🔍 Scraping detail 10/1138: SHE - Officer
🔍 Scraping detail 11/1138: Corporate Planning - Analyst
🔍 Scraping detail 12/1138: Strategic Planning - Analyst
🔍 Scraping detail 13/1138: TikTok Shop by Tokopedia - Strategic Solutions, Seller Acquisition (SMB)
🔍 Scraping detail 14/1138: TikTok Shop by Tokopedia - SMB Strategist
🔍 Scraping detail 15/1138: Production Section - Head
🔍 Scraping detail 16/1138: Production Operator - Staff
🔍 Scraping detail 17/1138: Indonesia Strategic Project Lead - ShopeePay
🔍 Scraping

In [90]:
# Step 6 – Simpan CSV
df_detail = pd.DataFrame(job_detail_data)
df_detail.to_csv("dealls_jobs_tech.csv", index=False)
print("\n✅ Selesai! Data disimpan ke dealls_jobs_tech.csv")


✅ Selesai! Data disimpan ke dealls_jobs_tech.csv


In [91]:
df_detail.tail(20)

,Kategori,Job Title,Company,Work Type,Work Mode,Experience,Salary,Detail Link,Job Description,Qualification
1118,Marketing & Social Media,Digital Ads Specialist,PT Qtera Mandiri,Penuh waktu,On-site • Jakarta Utara,Min. Freshgrad,Negotiable,https://dealls.com/loker/digital-ads-specialis...,Tanggung Jawab:,"Minimal S1 dari jurusan Marketing, Komunikasi,..."
1119,Marketing & Social Media,Social Media Specialist (Surabaya),Vista Education Corp.,Penuh waktu,On-site • Surabaya,Min. Freshgrad,Rp5.000.000 – 5.500.000,https://dealls.com/loker/social-media-speciali...,Requirements:,Copywriting: Ability to create engaging captio...
1120,Marketing & Social Media,Social Media Specialist,Evolene Group,Penuh waktu,On-site • Tangerang,Min. 1 years of experience,Rp5.000.000 – 7.000.000,https://dealls.com/loker/social-media-speciali...,Assist with content for social media and marke...,Atleast 1 year of experience.\nGood at talking...
1121,Marketing & Social Media,Marketing Admin,Delta Systech Indonesia,Kontrak,On-site • Tangerang Selatan,Min. Freshgrad,Negotiable,https://dealls.com/loker/marketing-admin-2~del...,"Merapikan, mengarsipkan, dan mengelola dokumen...",Pendidikan minimal SMK atau sederajat (diutama...
1122,Marketing & Social Media,Marketing Intern,UENA Food,Magang,On-site • Jakarta Barat,Freshgrad,Rp1.500.000 – 2.000.000,https://dealls.com/loker/marketing-intern-15~u...,Brand Awareness campaign both offline and onli...,Bachelors degree in Business and/or Management...
1123,Marketing & Social Media,Social Media Specialist,Felaufee,Penuh waktu,On-site • Jakarta Utara,Min. 1 years of experience,Rp6.000.000 – 9.000.000,https://dealls.com/loker/social-media-speciali...,Develop visually engaging and high-quality con...,"1+ year of experience as a content creator, wi..."
1124,Marketing & Social Media,Supervisor Digital Marketing & Sales,PT. Kira Kreasi Warna Indonesia,Penuh waktu,On-site • Surabaya,Min. 4 years of experience,Rp5.000.000 – 7.000.000,https://dealls.com/loker/supervisor-digital-ma...,Memimpin team digital Marketing & sales\nMembu...,minimal S1 Marketing / manajemen bisnis atau s...
1125,Marketing & Social Media,Social Media Specialist,Surprice Living,Penuh waktu,On-site • Jakarta Selatan,Min. 1 years of experience,Rp3.500.000 – 5.000.000,https://dealls.com/loker/social-media-speciali...,Employment Status: Full-time WFO\nLocation: Me...,Has 2+ years of experience managing social med...
1126,Marketing & Social Media,Host Live Streaming,PT GLOBAL JAYA ULTIMA,Penuh waktu,On-site • Tangerang Selatan,Min. 1 years of experience,Rp4.800.000 – 7.000.000,https://dealls.com/loker/host-live-streaming-6...,Membawakan sesi live streaming secara aktif da...,"Usia maksimal 30 tahun, berpenampilan menarik ..."
1127,Marketing & Social Media,KOL Specialist,Kavela,Penuh waktu,On-site • Bekasi,Min. 1 years of experience,Negotiable,https://dealls.com/loker/kol-specialist-55~kav...,Develop KOL/influencer campaign strategies to ...,"Minimum Bachelor's degree in Communications, M..."


In [92]:
df_detail.shape

(1138, 10)

In [93]:
# Step 5 - Close Browser
#driver.quit()
